In [39]:
import pandas as pd
import numpy as np

data=pd.read_csv(r'C:\Users\Reyan\Desktop\Proj\Apple\Data.csv')
data=data.sample(frac=1).reset_index(drop=True)

train_input_data=data.drop(['O'],axis=1)
train_input=np.matrix(train_input_data.values)
train_output_data=data.drop(['A','B','C','D'],axis=1)
train_output=np.matrix(train_output_data.values)
train_input[0].shape

(1, 4)

In [59]:
#set nn structure [input nodes,x hiddenlayers with x nodes....,output nodes, o for offset]
structure=[4,3,2,1,0]

#function to generate random chromosomes
def chromosome_generetor(structure):
    x=[]
    for i in range(0,len(structure)-1):
        k=structure[i]*structure[i+1]
        x.append(k)
    x=sum(x)
    # x = shape of chromsome
    return 2*np.random.random((x,))-2 

#function to arrange the weights for each layers 
def weights_classifier(structure,chromosome):
    weights_list=[]
    for i in range(0,len(structure)-2):
        if i == 0:
            a=0
            b=structure[i]*structure[i+1]
        else:
            a=b
            b=a+(structure[i]*structure[i+1])
        r=structure[i]
        c=structure[i+1]
        #print('[',a,',',b,']','(',r,',',c,')')
        w=np.reshape(np.matrix(chromosome[a:b]),(r,c))
        weights_list.append(w)
        
    # weights_list = a list of aranged weights matrices  
    return weights_list

def sigmoid(x):
    return 1/(1+np.exp(-x))

def feed_forward(structure,chromosome,train_input):
    weights_list=weights_classifier(structure,chromosome)
    #print('weights list: ',weights_list)
    for i in range(len(weights_list)):
        if i==0:
            output=sigmoid(np.dot(train_input,weights_list[i]))
        else:
            output=sigmoid(np.dot(output,weights_list[i]))
        #print('hl',[i],': ',weights_list[i])
    #print(output.shape)        
    return output
    

In [68]:
structure=[4,3,2,1,0]
chromosome=chromosome_generetor(structure)
for i in range(len(train_input)):
    output=feed_forward(structure,chromosome,train_input[i])
    #print(structure)
    print('output: ',output)

output:  [[0.44829578]]
output:  [[0.37515509]]
output:  [[0.40104461]]
output:  [[0.37184057]]
output:  [[0.32715311]]
output:  [[0.32347751]]
output:  [[0.3248758]]
output:  [[0.33380318]]
output:  [[0.30484559]]
output:  [[0.28670077]]
output:  [[0.30362034]]
output:  [[0.29467063]]
output:  [[0.39187244]]
output:  [[0.34632959]]
output:  [[0.29972842]]
output:  [[0.32847212]]


In [65]:
structure=[4,4,4,1,0]
chromosome=chromosome_generetor(structure)
for i in range(len(train_input)):
    output=feed_forward(structure,chromosome,train_input[i])
    #print(structure)
    print('output: ',output)

output:  [[0.41962001]]
output:  [[0.38160475]]
output:  [[0.3582655]]
output:  [[0.35785551]]
output:  [[0.32136635]]
output:  [[0.30074342]]
output:  [[0.31824609]]
output:  [[0.30316092]]
output:  [[0.26627595]]
output:  [[0.25149367]]
output:  [[0.26944252]]
output:  [[0.27677354]]
output:  [[0.33086415]]
output:  [[0.28281358]]
output:  [[0.2576655]]
output:  [[0.281456]]
